In [1]:
from stock_env.algos.agent import Agent
from stock_env.envs import *
from stock_env.common.common_utils import open_config
from stock_env.common.env_utils import make_vec_env
import torch as th
from stock_env.common.common_utils import create_performance, plot_trade_log_v2
from stock_env.common.evaluation import play_an_episode, evaluate_agent
from stock_env.common.common_utils import open_config
import seaborn as sns
sns.set()
import plotly.graph_objects as go
import os
from empyrical import (
    sharpe_ratio,
    max_drawdown,
)

In [2]:
data = pd.read_csv('../temp/eval_df.csv')
data

,Ticker,Algorithm,Ith,Sharpe,MaxDD
0,ACB,PPO,0,0.564580,6.123434
1,ACB,PPO,1,0.807212,18.443911
2,ACB,PPO,2,-0.629023,21.948539
3,ACB,PPO,3,0.564580,6.123434
4,ACB,PPO,4,0.564580,6.123434
...,...,...,...,...,...
828,VRE,Buy-n-Hold,5,-0.210338,25.628141
829,VRE,Buy-n-Hold,6,0.133817,29.722222
830,VRE,Buy-n-Hold,7,0.133817,29.722222
831,VRE,Buy-n-Hold,8,-0.210338,25.628141


In [18]:
grouped_data = data.groupby(['Algorithm', 'Ticker']).agg(['mean', 'std'])

In [19]:
grouped_data.loc['PPO']

Ith              Sharpe                MaxDD           
       mean       std      mean       std       mean        std
Ticker                                                         
ACB     4.5  3.027650  0.275367  0.640786  11.859249   8.028023
BID     4.5  3.027650  0.191920  0.453278  14.327632   9.388245
BVH     4.5  3.027650 -0.220797  0.651305  19.385031  10.584035
CTG     4.5  3.027650  0.727407  1.212601  18.796757   8.759495
FPT     4.5  3.027650  2.273633  1.791374   7.854680   5.437763
GAS     4.5  3.027650  0.282979  1.321071  10.967541   3.068143
GVR     4.5  3.027650 -0.772467  0.804391  35.117003  16.257585
HDB     4.5  3.027650  0.538715  1.416380  16.460892   9.689446
HPG     4.5  3.027650  0.621644  2.143168  18.227158  10.693408
MBB     4.5  3.027650  1.282317  1.231667  12.292633   7.159909
MSN     4.5  3.027650  0.475949  0.668627  13.373581   7.580643
MWG     4.5  3.027650  0.524438  0.829042  18.860449  12.211599
NVL     4.5  3.027650 -3.329864  6.129364  22.000399  21.049095
PDR     4.5  3.027650 -0.310534  3.228365  16.585701  12.790850
PLX     4.5  3.027650 -0.521686  1.290814  21.734539  14.788530
POW     4.5  3.027650 -0.379879  0.915236  24.450925   6.065430
SAB     4.5  3.027650 -0.423409  0.622613  18.426208  14.048097
SSI     4.5  3.027650  1.118231  1.833018  19.953630  12.486728
STB     4.5  3.027650  2.306110  2.513604  13.004394   8.348449
TCB     4.5  3.027650 -0.293891  1.265082  20.183234  13.597723
TPB     4.5  3.027650 -0.014748  1.919723  21.626046  11.750550
VCB     4.5  3.027650  0.977247  0.917442  10.694709   6.382330
VHM     4.5  3.027650 -1.052880  0.579812  21.330232   6.094631
VIB     4.5  3.027650  0.812229  0.843024  14.489963   8.185076
VIC     4.5  3.027650 -0.498950  0.875012  19.036607   7.597496
VJC     5.0  3.391165 -0.605536  0.399337  14.257307   2.270235
VNM     4.5  3.027650 -0.009319  0.466351  11.539464   5.152662
VPB     4.5  3.027650  0.055972  1.048284  16.680517   2.117328
VRE     4.5  3.027650 -0.237591  0.233615  14.843833   1.187416

In [25]:
fig = go.Figure()
fig.update_layout(width=1000, height=500)
fig.update_layout(title=f"Mean evaluation reward over every stock in VNALL", title_x=0.5)
# fig.update_xaxes(title_text="Number of learning steps before evaluation")
fig.update_yaxes(title_text="Sharpe")
# fig.update_layout(legend=dict(
#     yanchor="top",
#     y=0.99,
#     xanchor="left",
#     x=0.01
# ))
# fig.update_layout(barmode='group')
for algo in data['Algorithm'].unique():
    fig.add_trace(go.Bar(x=grouped_data.loc[algo].index, y=grouped_data.loc[algo]['Sharpe']['mean'], name=algo))
fig.show()

In [14]:
data[data.Algorithm == 'PPO']#['Sharpe']

,Ticker,Algorithm,Ith,Sharpe,MaxDD
0,ACB,PPO,0,0.564580,6.123434
1,ACB,PPO,1,0.807212,18.443911
2,ACB,PPO,2,-0.629023,21.948539
3,ACB,PPO,3,0.564580,6.123434
4,ACB,PPO,4,0.564580,6.123434
...,...,...,...,...,...
808,VRE,PPO,5,-0.418355,13.929178
809,VRE,PPO,6,0.024193,16.324103
810,VRE,PPO,7,0.024193,16.324103
811,VRE,PPO,8,-0.418355,13.929178
